<a href="https://colab.research.google.com/github/sandhyaparna/NLP/blob/main/Take%20Home%20Assignments/Dell_Take_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load tweets, replacements dictionary and important features files

In [ ]:
from google.colab import files

print("upload boston_bombing_tweets - Home test Data Scientist.csv file")
tweets_uploaded = files.upload()

upload boston_bombing_tweets - Home test Data Scientist.csv file


In [ ]:
print("upload replacements_dict.json file")
dict_uploaded = files.upload()

upload replacements_dict.json file


In [ ]:
print("upload important_features_ngrams.txt file")
imp_features_uploaded = files.upload()

In [ ]:
# import Libraries
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
from google.colab import files
import json
import seaborn as sns
import matplotlib.pyplot as plt
import string, re, os, io
import collections
from collections import defaultdict, Counter, OrderedDict

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
more_stopwords = ['i']
stop_words = stop_words + more_stopwords

import spacy
nlp = spacy.load('en', parse = False, tag=False, entity=False)

from wordcloud import WordCloud
import textblob
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, KFold, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, classification_report
from sklearn.naive_bayes import MultinomialNB

In [ ]:
boston_bombing_tweets = pd.read_csv(io.BytesIO(tweets_uploaded['boston_bombing_tweets - Home test Data Scientist.csv']))
print(f'Tweets data is loaded')

tweets_train = boston_bombing_tweets[pd.notnull(boston_bombing_tweets.label)]
print("Number of observations in train data are", tweets_train.shape[0])

tweets_test = boston_bombing_tweets[pd.isnull(boston_bombing_tweets.label)]
print("Number of observations in test data are", tweets_test.shape[0])

In [ ]:
# load replacements_dict
with open('replacements_dict.json') as json_file:
  replacements_dict = json_file.readlines()
  
replacements_dict = map(lambda x: x.rstrip(), replacements_dict)
replacements_dict = ' '.join(replacements_dict)
replacements_dict = json.loads(replacements_dict, object_pairs_hook=OrderedDict)
print("replacements dictionary is loaded")

In [ ]:
# load important_features_ngrams
important_features_ngrams = []

with open('important_features_ngrams.txt', 'r') as filehandle:
    for line in filehandle:
        feature = line[:-1]
        important_features_ngrams.append(feature)
print("important_features_ngrams is loaded")

In [ ]:
# helper functions
def replace_all(text, dic=replacements_dict):
    for i, j in dic.items():
        text = re.sub(i, j, text)
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text
    
def text_preprocess(text):
  text = re.sub('[%s]' % re.escape(string.punctuation.replace('#','')), '', text)
  text = re.sub('^# |# ', '', text)
  text = re.sub('[^\u0000-\u007e]+', '', text)
  text = ' '.join(word for word in text.split(' ') if word not in stop_words)
  return text

def create_corpus(data, text_var, condition):
    corpus=[]
    
    if condition == "hash":
      for x in data['text_processed'].apply(lambda x: re.findall(r'(?<=)#\w+',x)):
          for i in x:
            corpus.append(i)
    
    else:  # non-hash words
      for x in data['text_processed'].apply(lambda x: re.findall(r'(?#)\w+',x)):
        for i in x:
          corpus.append(i)

    return corpus

def get_top_tweet_ngrams(corpus, n):
    vec = CountVectorizer(ngram_range=(n, n), token_pattern=r'\b\w\w+\b|(?<!\w)#\w+').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:None]

In [ ]:
# pre-processing function
def preprocessing_fn(data, text_var):
  """
  delete missing values, 

  """
  data = data[pd.notnull(data[text_var])]
  data[text_var+'_processed'] = data[text_var].str.lower()
  data[text_var+'_URL_present'] = np.where(data[text_var+'_processed'].str.contains("http://"),1,0)
  data[text_var+'_retweet'] = np.where(data[text_var+'_processed'].str.contains("rt @"),1, 0)

  data[text_var+'_processed'] = data[text_var+'_processed'].apply(lambda x: replace_all(x,replacements_dict))
  data[text_var+'_punctuations_count'] = data[text_var+'_processed'].apply(lambda x: re.findall('[%s]' % re.escape(string.punctuation),str(x))).str.len() 
  data[text_var+'_word_count'] = data[text_var+'_processed'].apply(lambda x: len(str(x).split()))
  data[text_var+'_hashtag_count'] = data[text_var+'_processed'].apply(lambda x: len([c for c in str(x) if c == '#']))
  
  data[text_var+'_processed'] = data[text_var+'_processed'].apply(lambda x: lemmatize_text(x))
  data[text_var+'_processed'] = data[text_var+'_processed'].apply(lambda x: text_preprocess(x))

  return data

### Baseline Model

In [ ]:
# feature engineering of n-grams
def feature_engineering(data):
  features = []

  for i, j in Counter(create_corpus(data, "text_processed", "non-hash")).items():
    if j > 30:
      features.append(i)

  for i, j in Counter(create_corpus(data, "text_processed", "hash")).items():
    if j > 10:
      features.append(i)

  for i, j in get_top_tweet_ngrams(data["text_processed"], 2):
    if j > 10:
      features.append(i)

  for i, j in get_top_tweet_ngrams(data["text_processed"], 3):
    if j > 5:
      features.append(i)

  # print(features)
  # print(len(features))

  for i in features:
    data[i] = np.where(data["text_processed"].str.contains(i),1,0)  # data[i+"_ngram"]
  
  # data.columns = data.columns.str.replace(' ', '_')
  
  return data

In [ ]:
# model building
def build_model(data):
  
  data = data.drop(['text', 'text_processed'],axis=1)

  # split data into 80% Train & 20% Test - Stratified split
  X = data.drop(['label'],axis=1)
  y = data[['label']]
  X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.20, random_state=42)
  
  cv = KFold(n_splits=10, random_state=1, shuffle=True)
  # create model
  nb = MultinomialNB()
  # evaluate model
  scores = cross_val_score(nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # report performance
  print('Cross Validation Accuracy Score: %.3f (%.3f)\n' % (np.mean(scores), np.std(scores)))

  # train the model
  nb.fit(X_train, y_train)

  # make class and probability predictions
  y_pred_class = nb.predict(X_eval)
  y_pred_prob = nb.predict_proba(X_eval)[:, 1]

  print("Performance evaluation of the Naive Bayes model")

  print("Confusion Matrix","\n",pd.crosstab(y_eval.label, y_pred_class, rownames=['True'], colnames=['Predicted'], margins=True))

  print("\n","Classification Report","\n",classification_report(y_eval, y_pred_class))

  fpr, tpr, thresholds = roc_curve(y_eval, y_pred_prob)
  # create plot
  plt.plot(fpr, tpr, label='ROC curve'+', AUC='+str(round(roc_auc_score(y_eval, y_pred_prob).mean(),3)))
  plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
  _ = plt.xlabel('False Positive Rate')
  _ = plt.ylabel('True Positive Rate')
  _ = plt.title('ROC Curve')
  _ = plt.xlim([-0.02, 1])
  _ = plt.ylim([0, 1.02])
  _ = plt.legend(loc="lower right")

  return plt

In [ ]:
build_model(feature_engineering(preprocessing_fn(tweets_train, "text")))

### Model with only important features

In [ ]:
# Extract important features
imps = permutation_importance(nb, X_eval, y_eval)
importances = imps.importances_mean
std = imps.importances_std
indices = np.argsort(importances)[::-1]

independent_cols = list(X_eval)
important_features_ngrams = []
for f in range(X_eval.shape[1]):
  if importances[indices[f]] > 0: 
    important_features_ngrams.append(independent_cols[indices[f]])

len(important_features_ngrams)
important_features_ngrams[:5]

with open('important_features_ngrams.txt', 'w') as filehandle:
    for feature in important_features_ngrams:
        filehandle.write('%s\n' % feature)

In [ ]:
# imp = []
# for i in important_features:
#   i = i.replace("_ngram", "")
#   i = i.replace("_", " ")
#   imp.append(i)
# imp

In [ ]:
def feature_engineering_imp(data, important_features):

  for i in important_features:
    data[i] = np.where(data["text_processed"].str.contains(i),1,0)
  
  return data

In [ ]:
build_model(feature_engineering_imp(preprocessing_fn(tweets_train, "text"), important_features_ngrams))

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDFNGrams_vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=100)
X = TFIDFNGrams_vectorizer.fit_transform(tweets_train_processed.text_processed.values)
TextDf_TFIDFNGramsVectors = pd.DataFrame(X.A, columns=TFIDFNGrams_vectorizer.get_feature_names())
print(TextDf_TFIDFNGramsVectors.shape)
TextDf_TFIDFNGramsVectors.head()

In [ ]:
TextDf_TFIDFNGramsVectors_test = TFIDFNGrams_vectorizer.transform(tweets_test.text.values)